# Scrape and process @jongraz stats for Noodle videos

In [1]:
from TikTokApi import TikTokApi
import pandas as pd
import numpy as np
import altair as alt

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Read in our hand collected list of Noodle videos

In [3]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQz8eEafXnIeKsNvGL8M56Ia6vu89JDCjfJ22ORl65So4kRYtmTbwmqLjE223fe4mjcZqkrs-KT6j8z/pub?\
gid=0&single=true&output=csv"
)

### Create an ID field from the URL

In [4]:
src['id'] = src['url'].str.replace('https://www.tiktok.com/@jongraz/video/', "", regex=False).astype(str)

### Make a list for subsetting the dataframe later

In [5]:
bones_videos = src['id'].to_list()

### Have to run this manually -- for now

In [6]:
# Get cookie and run this...

In [7]:
# !python '/Users/stiles/github/noodle-tracker/_workspace/grab_user_stats.py'

---

### Read scraped stats data

In [8]:
df = pd.read_csv('../../tiktok/jongraz_videos.csv', dtype={'video_id':str})

### Clean update dates, times

In [9]:
df['timestamp'] = pd.to_datetime(df['video_time'], unit='s')
df['date'] = pd.to_datetime(df['video_time'], unit='s').dt.date
df['month'] = pd.to_datetime(df['video_time'], unit='s').dt.month
df['weekday'] = pd.to_datetime(df['video_time'], unit='s').dt.day_name()
df['time'] = pd.to_datetime(df['video_time'], unit='s').dt.time
df['weekstart_timestamp'] = pd.to_datetime(df['video_time'], unit='s') - pd.to_datetime(df['video_time'], unit='s').dt.weekday * np.timedelta64(1, 'D')
df['weekstart'] = df['weekstart_timestamp'].dt.date
df['updated_date'] = pd.to_datetime("today")

### How many videos? 

In [10]:
len(df)

155

### Select only "bones" videos from @jongraz's feed

In [11]:
bones_df = df[df['video_id'].isin(bones_videos)]

### How many are Noodle videos? 

In [12]:
len(bones_df)

66

In [13]:
bones_df.head()

,video_cover,user_name,user_id,video_id,video_desc,video_time,video_length,video_link,n_likes,n_shares,n_comments,n_plays,timestamp,date,month,weekday,time,weekstart_timestamp,weekstart,updated_date
0,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/5d44fb68b19240f99d6e68c59a4a6361?x-expires=1640808000&x-signature=Xmju68NhD3eHejTWzMR7lkN597A%3D,jongraz,6867588218972455941,7046779675493895430,plan or adjust your day accordingly 🔮🦴🔮 #noodletok #bonesday #nobones #magicmeatball,1640706249,55,https://www.tiktok.com/@jongraz/video/7046779675493895430?lang=en,93000,3116,1185,389000,2021-12-28 15:44:09,2021-12-28,12,Tuesday,15:44:09,2021-12-27 15:44:09,2021-12-27,2021-12-29 06:02:18.116093
2,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/464b4b9109d3491a80b90424ab896a33?x-expires=1640808000&x-signature=2X3%2BiRYPMwfw%2Bg9NZC8T8ColKMw%3D,jongraz,6867588218972455941,7045657418432285957,have an amazing day everyone and merry christmas to all who celebrate! 🔮🦴🎄 #noodletok #christmasbones #pug,1640444953,59,https://www.tiktok.com/@jongraz/video/7045657418432285957?lang=en,267700,14100,3084,1000000,2021-12-25 15:09:13,2021-12-25,12,Saturday,15:09:13,2021-12-20 15:09:13,2021-12-20,2021-12-29 06:02:18.116093
3,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/3fa0f6ff4d7e41e483d786ed0ee15f6e_1640274543?x-expires=1640808000&x-signature=kRDDLCZNwnns3t%2FBK%2FSQI4OgiOg%3D,jongraz,6867588218972455941,7044925508760407302,it’s a…snow bones day 😉😉😉😉 #noodletok #iceking #kingofwinter,1640274542,58,https://www.tiktok.com/@jongraz/video/7044925508760407302?lang=en,199000,8593,2166,808200,2021-12-23 15:49:02,2021-12-23,12,Thursday,15:49:02,2021-12-20 15:49:02,2021-12-20,2021-12-29 06:02:18.116093
4,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/b8891b9bcc8d4543a27bfbaba9a305eb?x-expires=1640808000&x-signature=bocJBta4OQojMl2jLIE8UDFfHPw%3D,jongraz,6867588218972455941,7044174342799789318,the Oracle Speaks and we must Listen!! 🔮🦴🔮 #noodletok #pug #rescue,1640099647,44,https://www.tiktok.com/@jongraz/video/7044174342799789318?lang=en,109500,4078,1714,526300,2021-12-21 15:14:07,2021-12-21,12,Tuesday,15:14:07,2021-12-20 15:14:07,2021-12-20,2021-12-29 06:02:18.116093
5,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/40635b5e839e48ab94c3615595388c39?x-expires=1640808000&x-signature=oUOkt7KTed5SmLIWbvODb4ld4QQ%3D,jongraz,6867588218972455941,7043809322018557189,your Monday reading 🔮🦴🔮 #noodletok #nobones,1640014659,13,https://www.tiktok.com/@jongraz/video/7043809322018557189?lang=en,251800,7333,2721,1200000,2021-12-20 15:37:39,2021-12-20,12,Monday,15:37:39,2021-12-20 15:37:39,2021-12-20,2021-12-29 06:02:18.116093


---

### Download cover art

In [14]:
# for v,u in zip(bones_df['video_cover'], bones_df['video_id']):
#     !wget {v} -O ../assets/images/noodle_covers/{u}

---

### Weekly plays

In [15]:
weekly_plays = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_plays':sum}).reset_index()
weekly_plays.dtypes

weekstart    object
n_plays       int64
dtype: object

In [16]:
weekly_plays['weekstart'] = pd.to_datetime(weekly_plays['weekstart'])

In [17]:
alt.Chart(weekly_plays).mark_bar(width=30).encode(
    x='weekstart:T',
    y='n_plays'
).properties(width=630)

alt.Chart(...)

---

### Weekly likes

In [18]:
weekly_likes = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_likes':sum}).reset_index()
weekly_likes.dtypes

weekstart    object
n_likes       int64
dtype: object

In [19]:
weekly_likes['weekstart'] = pd.to_datetime(weekly_likes['weekstart'])

In [20]:
alt.Chart(weekly_likes).mark_bar(width=30).encode(
    x='weekstart:T',
    y='n_likes'
).properties(width=630)

alt.Chart(...)

---

### Weekly comments

In [21]:
weekly_comments = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_comments':sum}).reset_index()
weekly_comments.dtypes

weekstart     object
n_comments     int64
dtype: object

In [22]:
weekly_comments['weekstart'] = pd.to_datetime(weekly_comments['weekstart'])

In [23]:
alt.Chart(weekly_comments).mark_bar(width=30).encode(
    x='weekstart:T',
    y='n_comments'
).properties(width=630)

alt.Chart(...)

---

### Exports

In [24]:
weekly_plays.to_csv("../_data/weekly_plays.csv", index=False)
weekly_plays.to_csv("../assets/data/weekly_plays.csv", index=False)

In [25]:
weekly_likes.to_csv("../_data/weekly_likes.csv", index=False)
weekly_likes.to_csv("../assets/data/weekly_likes.csv", index=False)

In [26]:
weekly_comments.to_csv("../_data/weekly_comments.csv", index=False)
weekly_comments.to_csv("../assets/data/weekly_comments.csv", index=False)